In [ ]:
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

from dask.distributed import Client

from carbonplan.data import cat as core_cat

from cmip6_downscaling.data import cat
from cmip6_downscaling.workflows.share import get_cmip_runs

from carbonplan_styles.mpl import set_theme

In [ ]:
set_theme(style="carbonplan_light")

In [ ]:
mapper = fsspec.get_mapper(
    "az://carbonplan-scratch/cmip6/quantile-mapping/conus/4000m/monthly/ACCESS-ESM1-5.ssp370.r10i1p1f1.zarr",
    account_name="carbonplan",
)

test = xr.open_zarr(mapper, consolidated=True)
test_ts = test.mean(("x", "y")).load()
test_ts

In [ ]:
grid = core_cat.grids.conus4k.to_dask()

In [ ]:
grid.mask.plot()

In [ ]:
client = Client(threads_per_worker=1)
client

Get a table of processed model runs.


In [ ]:
df = get_cmip_runs()

Pull each run into a Xarray dataset


In [ ]:
data = {}
index = []
for method in ["quantile-mapping", "bias-corrected"]:
    for i, row in df.iterrows():
        key = f"{method}.{row.model}.{row.scenario}.{row.member}"
        data[key] = cat.cmip6(
            method=method,
            model=row.model,
            scenario=row.scenario,
            member=row.member,
        ).to_dask()
        index.append((method, row.model, row.scenario))

In [ ]:
grid = grid.chunk({"x": 50, "y": 50}).persist()
obs_ts = cat.obs.to_dask().where(grid.mask).mean(("x", "y")).load()

In [ ]:
ann_ts = {}
for k, ds in data.items():
    ann_ts[k] = ds.mean(("x", "y")).load()

In [ ]:
def multi_index_from_keys(keys):
    return pd.MultiIndex.from_tuples(
        [tuple(k.split(".")[:3]) for k in keys],
        names=["method", "model", "scenario"],
    )


def combine(data):
    dim = xr.Variable("run", multi_index_from_keys(data.keys()))
    ds = xr.concat(data.values(), dim=dim, coords="minimal", compat="override")
    ds = ds.unstack(dim="run")
    ds["method"] = ds["method"].astype(str)
    ds["model"] = ds["model"].astype(str)
    ds["scenario"] = ds["scenario"].astype(str)
    return ds

In [ ]:
hist_ds = combine({k: ds for k, ds in ann_ts.items() if "hist" in k})
ssp_ds = combine({k: ds for k, ds in ann_ts.items() if "ssp" in k})

In [ ]:
ssp_ds

In [ ]:
hist_method_diff = hist_ds.isel(method=1) - hist_ds.isel(method=0)
ssp_method_diff = ssp_ds.isel(method=1) - ssp_ds.isel(method=0)
hist_method_diff

In [ ]:
methods = hist_ds["method"].values
var_names = ["tmean", "ppt", "pet", "def", "soil", "vpd", "pdsi"]
colors = {"ssp245": "yellow", "ssp370": "orange", "ssp585": "red"}

fig, axes = plt.subplots(
    nrows=len(var_names), ncols=len(methods) + 1, figsize=(12, 12), sharex=True
)

for i, var in enumerate(var_names):
    for j, method in enumerate(methods):
        hist_ds[var].isel(method=j).squeeze(drop=True).resample(
            time="1AS"
        ).mean().rolling(time=1).mean().plot.line(
            x="time", ax=axes[i, j], color="gray", alpha=0.8, add_legend=False
        )
        for scen in ssp_ds["scenario"].values:
            ssp_ds[var].isel(method=j).sel(scenario=scen).squeeze(
                drop=True
            ).resample(time="1AS").mean().rolling(time=1).mean().plot.line(
                x="time",
                ax=axes[i, j],
                color=colors[scen],
                alpha=0.8,
                add_legend=False,
            )
        obs_ts[var].squeeze(drop=True).resample(time="1AS").mean().rolling(
            time=1
        ).mean().plot.line(
            x="time", ax=axes[i, j], color="k", alpha=1, add_legend=False
        )
        try:
            test_ts[var].resample(time="1AS").mean().rolling(
                time=1
            ).mean().plot.line(
                x="time", ax=axes[i, 0], color="g", alpha=1, add_legend=False
            )
        except:
            pass
        axes[i, j].set_xlabel("")
        if j != 0:
            axes[i, j].set_ylabel("")
        if i == 0:
            axes[i, j].set_title(method)
        else:
            axes[i, j].set_title("")

    #     hist_method_diff[var].squeeze(drop=True).resample(time='1AS').mean().rolling(time=10).mean().plot.line(
    #         x='time', ax=axes[i, -1], color='gray', alpha=0.8, add_legend=False)
    #     for scen in ssp_ds['scenario'].values:
    #         ssp_method_diff[var].sel(scenario=scen).squeeze(drop=True).resample(time='1AS').mean().rolling(time=10).mean().plot.line(
    #             x='time', ax=axes[i, -1], color=colors[scen], alpha=0.8, add_legend=False)
    axes[i, -1].set_xlabel("")
    axes[i, -1].set_ylabel("")
    if i == 0:
        axes[i, -1].set_title("qm-bc")
    else:
        axes[i, -1].set_title("")


axes[-1, 0].set_ylim(-25, 5)
fig.tight_layout()

In [ ]:
hist_periods = [
    ("1970", "2000"),
]
ssp_periods = [("2020", "2050"), ("2060", "2090")]

maps = {}
for k, ds in data.items():
    maps[k] = {}
    periods = hist_periods if "hist" in k else ssp_periods
    for p in periods:
        kp = ":".join(p)

        maps[k][kp] = ds.sel(time=slice(*p)).mean(dim="time").load()

In [ ]:
obs_map = (
    cat.obs.to_dask().sel(time=slice(*hist_periods[0])).mean(dim="time").load()
)
obs_map

In [ ]:
maps[k][kp]

In [ ]:
hist_maps = combine({k: maps[k][pk] for k in maps.keys() if "hist" in k})
hist_maps["method"] = ["bc", "qm"]

In [ ]:
obs_map["ws"].plot(robust=True)

In [ ]:
var = "tmean"


def plot_hist(
    hist_maps,
    obs_map,
    var,
    vmin=None,
    vmax=None,
    vmind=None,
    vmaxd=None,
    cmap=None,
    cmapd=None,
):
    fig, axes = plt.subplots(
        ncols=5,
        nrows=hist_maps.dims["model"],
        sharex=True,
        sharey=True,
        figsize=(12, 12),
        constrained_layout=True,
    )

    axes[0, 4].set_title("-".join(hist_maps.method.values))

    for i, model in enumerate(hist_maps.model.values):
        for j, method in enumerate(hist_maps.method.values):
            if i == 0:
                axes[i, j].set_title(method)
                axes[i, j + 2].set_title(method + "-obs")
            if j == 0:
                axes[i, j].set_ylabel(model)

            left_cols = (
                hist_maps[var]
                .isel(model=i, method=j)
                .plot(
                    ax=axes[i, j],
                    add_colorbar=False,
                    vmin=vmin,
                    vmax=vmax,
                    cmap=cmap,
                    add_labels=False,
                )
            )

            bias = hist_maps[var].isel(model=i, method=j) - obs_map[var]
            right_cols = bias.plot(
                ax=axes[i, j + 2],
                add_colorbar=False,
                vmin=vmind,
                vmax=vmaxd,
                cmap=cmapd,
                add_labels=False,
            )

        method_diff = hist_maps[var].isel(model=i, method=1) - hist_maps[
            var
        ].isel(model=i, method=0)
        method_diff.plot(
            ax=axes[i, 4],
            add_colorbar=False,
            vmin=vmind,
            vmax=vmaxd,
            cmap=cmapd,
            add_labels=False,
        )

    fig.colorbar(left_cols, ax=axes[-1, :2], location="bottom", shrink=0.9)
    fig.colorbar(right_cols, ax=axes[-1, 2:], location="bottom", shrink=0.7)
    fig.suptitle(var, y=1.03, fontweight="bold")
    return fig, axes

In [ ]:
kwargs = {
    "tmean": {
        "vmin": 0,
        "vmax": 25,
        "vmind": -0.5,
        "vmaxd": 0.5,
        "cmap": "cividis_r",
        "cmapd": "RdBu_r",
    },
    "ppt": {
        "vmin": 0,
        "vmax": 200,
        "vmind": -1,
        "vmaxd": 1,
        "cmap": "Blues",
        "cmapd": "BrBG",
    },
    "pet": {
        "vmin": 0,
        "vmax": 150,
        "vmind": -25,
        "vmaxd": 25,
        "cmap": "cividis",
        "cmapd": "RdBu_r",
    },
    "def": {
        "vmin": 0,
        "vmax": 150,
        "vmind": -25,
        "vmaxd": 25,
        "cmap": "cividis",
        "cmapd": "RdBu_r",
    },
    "soil": {
        "vmin": 0,
        "vmax": 150,
        "vmind": -25,
        "vmaxd": 25,
        "cmap": "cividis",
        "cmapd": "RdBu_r",
    },
    "vpd": {
        "vmin": 0,
        "vmax": 1,
        "vmind": -0.25,
        "vmaxd": 0.25,
        "cmap": "cividis",
        "cmapd": "RdBu_r",
    },
    "pdsi": {
        "vmin": -0.5,
        "vmax": 0.5,
        "vmind": -0.25,
        "vmaxd": 0.25,
        "cmap": "cividis",
        "cmapd": "RdBu_r",
    },
    "srad": {
        "vmin": 140,
        "vmax": 220,
        "vmind": -1,
        "vmaxd": 1,
        "cmap": "cividis",
        "cmapd": "RdBu_r",
    },
    "tdew": {
        "vmin": -15,
        "vmax": 15,
        "vmind": -1,
        "vmaxd": 1,
        "cmap": "cividis",
        "cmapd": "RdBu_r",
    },
}
for var in [
    "tdew"
]:  # ['tmean', 'ppt', 'pet', 'def', 'soil', 'vpd', 'pdsi', 'srad', 'srad', ]:
    fig, axes = plot_hist(hist_maps, obs_map, var=var, **kwargs[var])